# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
import findspark
findspark.init('/home/yannick/dev/spark-2.4.4-bin-hadoop2.7')

In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [39]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 
spark = SparkSession \
    .builder \
    .appName("Dataframe QUIZ") \
    .getOrCreate()

In [40]:
path = "data/sparkify_log.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [41]:
# TODO: write your code to answer question 1
#user_log.describe('artist')
user_log.printSchema()
#user_log.select('artist').show()
user_log.select('artist').filter(user_log.userId== '').count()
user_log.select('page').filter(user_log.userId== '').dropDuplicates().show()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [42]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select('page','artist','userID').filter(user_log.userId =='').show()

+-----+------+------+
| page|artist|userID|
+-----+------+------+
|Login|  null|      |
| Home|  null|      |
| Home|  null|      |
| Home|  null|      |
|Login|  null|      |
|Login|  null|      |
| Home|  null|      |
| Home|  null|      |
|Login|  null|      |
| Home|  null|      |
|Login|  null|      |
| Home|  null|      |
| Help|  null|      |
| Home|  null|      |
|Login|  null|      |
| Home|  null|      |
|Login|  null|      |
| Home|  null|      |
|Login|  null|      |
| Home|  null|      |
+-----+------+------+
only showing top 20 rows



# Question 3

How many female users do we have in the data set?

In [64]:
user_log.select('userId','gender').dropDuplicates().filter(user_log.gender=='F').count()

462

# Question 4

How many songs were played from the most played artist?

In [60]:
# TODO: write your code to answer question 4
user_log.select('artist','song').groupBy('artist').agg({'artist':'count'}).withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')).show()

+--------------------+-----------+
|              artist|Artistcount|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
|       Justin Bieber|         43|
|      The Black Keys|         40|
|         OneRepublic|         37|
|        Jack Johnson|         36|
|                Muse|         36|
|           Radiohead|         31|
|        Taylor Swift|         29|
|          Lily Allen|         28|
|Barry Tuckwell/Ac...|         28|
|               Train|         28|
|          Nickelback|         27|
|           Daft Punk|         27|
|           Metallica|         27|
|          Kanye West|         26|
|     Alliance Ethnik|         24|
+--------------------+-----------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [65]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [93]:
flag_downgrade_event = udf(lambda x : 1 if x == 'Submit Downgrade' else 0, IntegerType()   )

In [94]:
user_log_valid = user_log.withColumn('downgraded',flag_downgrade_event('page'))

In [95]:
user_log_valid.take(2)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', downgraded=0),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000', downgraded=0)]

In [96]:
windowVal = Window.partitionBy('userId').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding,0)

In [97]:
user_log_valid = user_log_valid.withColumn('Phase' , Fsum('downgraded').over(windowVal))

In [102]:
user_log_valid.select('userId','firstName','page','phase','ts').where(user_log_valid.userId == 1138).sort('ts').collect()

[Row(userId='1138', firstName='Kelly', page='Home', phase=1, ts=1513729066284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513729066284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513729313284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513729552284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513729783284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513730001284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513730263284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513730518284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513730768284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513731182284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=1513731435284),
 Row(userId='1138', firstName='Kelly', page='NextSong', phase=1, ts=151373169528